In [1]:
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

import numpy as np

import pydrake.all
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector, PiecewiseQuaternionSlerp,
    RandomGenerator, UniformlyRandomRotationMatrix, Simulator, FixedOffsetFrame
    )

from panda_station import *

In [2]:
"""
builder = DiagramBuilder()
station = PandaStation()


names_and_links = [(name, "base_link") for name in parse_tables(find_resource("directives/three_tables.yaml"))]
station.setup_from_file("directives/three_tables.yaml", names_and_links = names_and_links)
station.add_panda_with_hand()
plant, scene_graph = station.get_plant_and_scene_graph()

#station.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.6, 0, 0.2]))
station.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.8, 0, 0.2]),
                           main_body_name = "base_link", welded = True, name = "A")
station.add_model_from_file(find_resource("models/manipulands/sdf/dumbbell.sdf"), RigidTransform(),
                           main_body_name = "base_link", P = plant.GetFrameByName("panda_hand"), welded = True,
                            name = "B"
                           )


station.finalize()

builder.AddSystem(station)
meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions
meshcat.load()
diagram = builder.Build()
simulator = Simulator(diagram)
simulator_context = simulator.get_context()
station_context = station.GetMyContextFromRoot(simulator_context)
station.GetInputPort("panda_position").FixValue(station_context, np.array([np.pi/4, 0.1, 0, -1, 0, 1.5, 0]))
station.GetInputPort("hand_position").FixValue(station_context, [0.08])

meshcat.start_recording()
simulator.AdvanceTo(1.0)

print("PARSING STATION POSES")
print(parse_start_poses(station, station_context))
print("PARSING INITAL CONFIG")
print(parse_config(station, station_context))
print("PARSE TABLES")
print(parse_tables(find_resource("directives/three_tables.yaml")))

pose_fluents = [
    ("grasppose", "B", RigidTransform(RotationMatrix(), [0, 0, 0.3])),
    ("atpose", "A", RigidTransform(RotationMatrix(), [0.8, 0.2, 0.2]))
]
update_station(station, station_context, pose_fluents)

simulator_context = simulator.get_context()
plant_context = plant.GetMyContextFromRoot(simulator_context)
station_context = station.GetMyContextFromRoot(simulator_context)

simulator.AdvanceTo(2.0)
meshcat.stop_recording()
meshcat.publish_recording()
"""

'\nbuilder = DiagramBuilder()\nstation = PandaStation()\n\n\nnames_and_links = [(name, "base_link") for name in parse_tables(find_resource("directives/three_tables.yaml"))]\nstation.setup_from_file("directives/three_tables.yaml", names_and_links = names_and_links)\nstation.add_panda_with_hand()\nplant, scene_graph = station.get_plant_and_scene_graph()\n\n#station.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.6, 0, 0.2]))\nstation.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.8, 0, 0.2]),\n                           main_body_name = "base_link", welded = True, name = "A")\nstation.add_model_from_file(find_resource("models/manipulands/sdf/dumbbell.sdf"), RigidTransform(),\n                           main_body_name = "base_link", P = plant.GetFrameByName("panda_hand"), welded = True,\n                            name = "B"\n                           )\n\n\ns

In [3]:
p = ProblemInfo("/home/agrobenj/drake-tamp/experiments/pick-place-regions/problems/test_problem.yaml")
print(p)

directive: directives/three_tables.yaml
object: foam_brick
X_WO: RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [-0.0, 0.0, 1.0],
  ]),
  p=[0.6, 0.0, 0.2],
)
object: meat_can
X_WO: RigidTransform(
  R=RotationMatrix([
    [1.0, -0.0, 0.0],
    [0.0, 0.0007963267107332633, 0.9999996829318346],
    [-0.0, -0.9999996829318346, 0.0007963267107332633],
  ]),
  p=[0.5, 0.2, 0.2],
)
object: soup_can
X_WO: RigidTransform(
  R=RotationMatrix([
    [1.0, -0.0, 0.0],
    [0.0, 0.0007963267107332633, 0.9999996829318346],
    [-0.0, -0.9999996829318346, 0.0007963267107332633],
  ]),
  p=[0.5, -0.2, 0.2],
)



In [5]:
stations = p.make_all_stations()
print(list(stations.keys()))
builder = DiagramBuilder()
station = stations["meat_can"]
builder.AddSystem(station)
plant, scene_graph = station.get_plant_and_scene_graph()
meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions
meshcat.load()
diagram = builder.Build()
simulator = Simulator(diagram)
simulator_context = simulator.get_context()
station_context = station.GetMyContextFromRoot(simulator_context)
station.GetInputPort("panda_position").FixValue(station_context, np.array([np.pi/4, 0.1, 0, -1, 0, 1.5, 0]))
station.GetInputPort("hand_position").FixValue(station_context, [0.08])

meshcat.start_recording()
simulator.AdvanceTo(2.0)
meshcat.stop_recording()
meshcat.publish_recording()

['main', 'move_free', 'foam_brick', 'meat_can', 'soup_can']
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
